<h1 align="center"><font size="6"> Les méthodes de descente de gradient </font> (deuxième partie)</h1>
<hr> 

<h1>Table des matières</h1>

<div class="alert alert-block alert-info" style="margin-top : 20px">
      <ul>
          <li><a href="#prelim">Préliminaires</a></li>
          <li><a href="#Newton">La méthode de Newton</a></li>
          <li><a href="#BFGS">Une méthode de quasi-Newton (BFGS)</a></li>
      </ul>
</div>
<br>
<h>

<a id='prelim'></a>
<h2>Préliminaires</h2>
<hr>

On commence par importer les bibliothèques neecéssaires (_numpy_ et _matplotlib.pyplot_).

On définit aussi deux functions pour la visualisation de: *1/* les lignes de niveaux de la fonction ojectif et *2/* le champ de gradients (pour les fonctions objectifs dépendant de deux variables). 

Il y a aussi un exemples de graphique utilisé pour observer la vitesse de convergence des méthodes d'optimisation.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact
from mpl_toolkits.mplot3d import Axes3D

def draw_vector_field(F, xmin, xmax, ymin, ymax, N=15):
    X = np.linspace(xmin, xmax, N)  # x coordinates of the grid points
    Y = np.linspace(ymin, ymax, N)  # y coordinates of the grid points
    U, V = F(*np.meshgrid(X, Y))  # vector field
    M = np.hypot(U, V)  # compute the norm of (U,V)
    M[M == 0] = 1  # avoid division by 0
    U /= M  # normalize the u componant
    V /= M  # normalize the v componant
    return plt.quiver(X, Y, U, V, angles='xy')

def level_lines(f, xmin, xmax, ymin, ymax, levels, N=500):
    x = np.linspace(xmin, xmax, N)
    y = np.linspace(ymin, ymax, N)
    z = f(*np.meshgrid(x, y))
    level_l = plt.contour(x, y, z, levels=levels)
    #plt.clabel(level_l, levels, fmt='%.1f') 

f = lambda x, y : np.cosh(x)+ np.sin(x + y)**2
df = lambda x, y : np.array([np.sinh(x) + 2*np.cos(x + y)*np.sin(x + y), 2*np.cos(x + y)*np.sin(x + y)])
%matplotlib inline
level_lines(f, -1.1, 1.1, -1.1, 1.1, np.linspace(1, 3, 10))
draw_vector_field(df, -1.1, 1.1, -1.1, 1.1, 10)
plt.axis('equal')
plt.show()


# plot of the values of f along the iterations.
N = 10
F = 2**(-np.linspace(0,N,N+1))
plt.figure()
plt.semilogy(range(N + 1), F, '.', linestyle='dashed')

<a id='Newton'></a>
<h2>La méthode de Newton</h2>
<hr>

On suppose dans ce TP que $f:\mathbb{R}^N\to\mathbb{R}$ est de classe $C^2$ au moins.

La méthode de Newton (ou de Newton-Raphson) est une méthode de descente itérative dans laquelle la direction de descente à l'étape $k$ est choisie de manière à minimiser le développement limité au second ordre de $f$ au point $x^k$, c'est-à-dire
$$
\tag{4}
m_k(d):=f(x^k) + d\cdot \nabla f(x^k) + \dfrac12 d^T D^2 f(x^k) d.
$$
Si la matrice (symétrique) $D^2 f(x^k)$ est définie  positive le minimiseur de $m^k$ existe et est unique. On note $H^k$ l'inverse de $D^2 f(x^k)$, $g^k:=\nabla f(x^k)$ et $d^k$ le minimiseur de (4).

***Question 15.*** Exprimez $d^k$ en fonction de $H^k$ et $g^k$.

___Solution:___ $d^k=-H^kg^k$.

Lorsque la direction de descente $d^k$ a été calculée, nous procédons comme dans la méthode de descente de gradient et utilisons une méthode de backtracking ($\alpha \leftarrow \beta\alpha$) pour trouver $\alpha_k$ tel que $f(x^k+ \alpha_k d^k)$ soit  "suffisamment plus petit" que $f(x^k)$. Pour quantifier ce "suffisamment plus petit", nous utiliserons à nouveau le critère d'Armijo : 

$$
\tag{5}
f(x^k+\alpha_kd^k)\, \le\, f(x^k) + c\, \alpha_k \left<d^k;g^k\right>,
$$
où $c\in(0,1)$ sera fixé. On utilisera la même méthode de baEt la méthode

Ensuite, l'algorithme de Newton-Raphson avec une line search par critère d'Armijo se décrit comme ceci : 

Au départ, on fixe $c,\beta\in(0,1)$ et on choisit $x^0\in\mathbb{R}^N$.
<br> 
Ensuite pour $k=0,1,\dots$ jusqu'à convergence, on répète :
$$
\left|
\begin{array}{l}
\text{Calculer }d^k,\\
\text{Trouver }\alpha_k>0 \text{ tel que (5) soit vrai,}\\
\text{Faire }x^{k+1}\,\leftarrow\,x^k + \alpha_k d^k.
\end{array}
\right.
$$

Un point important ici est que, lorsque $x^k$ est suffisamment proche d'un minimiseur local $x^*$ de $f$ tel que $D^2f(x^*)$ soit défini positif, alors le choix $\alpha_k=1$ donne une convergence quadratique vers $x^*$, _i.e._ 
$$
\|x^{k+1}-x^*\|\leq C \|x^k - x^*\|^2.
$$
Si nous ne voulons pas perdre cette convergence super-linéaire, nous devons choisir $\alpha_k=1$ dès que $x^k$ est assez proche de $x^*$. Pour cela, nous commençons les itérations de back-tracking avec $\alpha=1$ et nous choisissons une valeur de $c$   relativement petite dans (5) (par exemple $c=0.1$).
<br>

Soit $\Lambda>0$. On pose $f_\Lambda(x,y):=(1-x)^2 + \Lambda\,(y-x^2)^2$, pour$(x,y)\in\mathbb{R}^2$. 

__Question 16.__ Calculez $\nabla f_\Lambda(x,y)$. Trouves le(s) minimiseur(s) de $f_\Lambda$. Tracez quelques lignes de niveau de $f_\Lambda$ ainsi que le champ vectoriel renormalisé $(1/|\nabla f_\Lambda|)\nabla f_\Lambda$ pour $\Lambda=100$. Calculez $D^2 f(x,y)$ et son inverse $H_\Lambda(x,y)$.

In [ ]:
Lambda = 1000
f = lambda x,y : ( x - 1)**2 + Lambda*(y - x**2)**2
df = lambda x,y : np.array([2*(x - 1) + 4*Lambda*x*(x**2 - y), 2*Lambda*(y - x**2)])
ddf = lambda x,y : np.array([[2 - 4*Lambda*y + 12*Lambda*x**2 , -4*Lambda*x], [-4*Lambda*x, 2*Lambda]])
HH = lambda x,y : (1/(4*Lambda - 8*(Lambda**2)*y 
                      + 8*(Lambda**2)*x**2))*np.array([[2*Lambda, 4*Lambda*x],
                                                       [4*Lambda*x, 2 - 4*Lambda*y + 12*Lambda*x**2 ]])

%matplotlib inline
level_lines(f, .8, 1.2, 0.8, 1.2, np.linspace(0, 30, 80))
draw_vector_field(df, .8, 1.2, 0.8, 1.2, 15)
plt.plot(1,1,'or')
plt.axis('equal')
plt.show()

__Question 17.__ Implémentez la méthode de Newton et appliquez-la à la fonction ci-dessus avec $c=0.1$, $\beta=0.75$ et $x^0=(0,0)$. Représentez les itérations sur un graphique et tracez $\ \log(f_\Lambda(x^k))\ $ en fonction de $k$. Commentez les résultats.

_Indication:_ Testez d'abord l'algorithme sur la fonction quadratique ci-dessous.

In [ ]:
# Pour le test
'''
f = lambda x,y : ( x - 1)**2 + 2*(y - 1)**2
df = lambda x,y : np.array([2*(x - 1) , 4*(y - 1)])
#ddf = lambda x,y : np.array([[2  , 0], [0, 2]])
HH = lambda x,y : np.array([[.5, 0], [0, .25]])
'''
Lambda = 1000
f = lambda x,y : ( x - 1)**2 + Lambda*(y - x**2)**2
df = lambda x,y : np.array([2*(x - 1) + 4*Lambda*x*(x**2 - y), 2*Lambda*(y - x**2)])
ddf = lambda x,y : np.array([[2 - 4*Lambda*y + 12*Lambda*x**2 , 
                              -4*Lambda*x],[-4*Lambda*x, 2*Lambda]])
HH = lambda x,y : (1/(4*Lambda - 8*(Lambda**2)*y 
                    + 8*(Lambda**2)*x**2))*np.array([[2*Lambda, 4*Lambda*x],
                    [4*Lambda*x, 2 - 4*Lambda*y + 12*Lambda*x**2 ]])

In [ ]:
## Parameters
c, beta = .1, .75
epsilon = 1e-13
itermax = 200
iter_ls_max = 40

## initialization 
x, y = 0.2, 0.5
w = np.array([x, y])
fw = f(x, y)
g = np.array(df(x, y))
ng=np.hypot(g[0], g[1])
ng0=ng
iteration = 0

## For the post-analysis
flag = 'OK'
W, F, NG, Alpha =[w], [fw], [ng], []

In [ ]:
## Optmization loop
while (iteration < itermax):
    iteration += 1
    d = -np.dot(HH(x, y), g)
    dx, dy = d[0], d[1]
    if ng < epsilon*ng0 or flag == 'Not OK':
        break
    mcdg, alpha = -c*np.dot(d,g), 1
    new_fw = f(x + alpha*dx, y + alpha*dy) 
    iter_ls = 0
    while (new_fw - fw + alpha*mcdg >=0):
        alpha *=beta
        new_fw = f(x + alpha*dx, y + alpha*dy)
        iter_ls += 1
        if (iter_ls>=iter_ls_max):
            flag = 'Not OK'
            break
    x, y, fw = x + alpha*dx, y + alpha*dy, new_fw
    g = np.array(df(x, y))
    ng=np.hypot(g[0], g[1])
    
    W.append(np.array([x, y]))
    F.append(fw)
    NG.append(ng)
    Alpha.append(alpha)

In [ ]:
print(f"flag = {flag}, nombre d'itérations = {iteration}")    
    
W = np.array(W)
F = np.array(F)

# plot the results 
plt.figure()
plt.plot(W[:,0],W[:,1],'.',linestyle='-')
level_lines(f, 0, 2, 0, 2, np.linspace(1, 3, 10))
draw_vector_field(df, 0 , 2, 0, 2, 10)
plt.title(r"Les itérés $w^k$", fontsize=15)
plt.axis('equal')
plt.show()

# plot of the values of f along the iterations.
plt.figure()
plt.semilogy(range(len(F)),F,'.',linestyle=':')
plt.title(r"$f(w^k)$ en fonction des itérations", fontsize=15)
plt.show()

# plot of the values of norm (grad f) along the iterations.
plt.figure()
plt.semilogy(range(len(NG)),np.array(NG)/ng0,'.',linestyle=':')
plt.title(r"$\|\nabla f(w^k)\|/\|\nabla f(w^0)\|$ en fonction des itérations", fontsize=15)
plt.show()

# plot of the values of the step alpha along the iterations.
plt.figure()
plt.plot(range(len(Alpha)),Alpha,'.',linestyle=':')
plt.title(r"$\alpha$ en fonction des itérations", fontsize=15)
plt.show()

__Commentaires__ : On observe bien les deux comportements de la méthode de Newton, d'abord une approche du minimiseur assez lente, ensuite une convergence extrèmement rapide, qui correspond à l'acceptation systématique du pas $\alpha=1$ par le critère d'Armijo.

<a id='BFGS'></a>
<h2> Une méthode de quasi-Newton (BFGS)</h2>
<hr>

Lorsque le nombre de paramètres est important comme il est habituel en Machine Learning, le calcul des matrices hessiennes $D^2f(x^k)$ et la résolution des systèmes linéaires $D^2f(x^k) d^k=-g^k$ peuvent être trop coûteux. Cependant, il est souvent encore possible d'obtenir une convergence superlinéaire en remplaçant $[D^2f(x^k)]^{-1}$ par une approximation moins gourmande à calculer qu'on notera $H^k$. Il existe plusieurs algorithmes basés sur cette idée. Nous présentons l'une des plus populaires : la méthode BFGS du nom de leurs découvreurs (Broyden, Fletcher, Goldfarb et Shanno). 

__Description de la méthode__ : Supposons qu'à l'étape $k$ nous ayons une approximation définie positive symétrique $H^k$ de $\left[D^2f(x^k)\right]^{-1}$. On note $B^k$ son inverse (qui est une approximation de $D^2f(x^k)$). Comme ci-dessus, nous définissons notre direction de descente $d^k$ comme le minimiseur de
$$
f(x^k) + d\cdot \nabla f(x^k) + \dfrac12 d^T B^k d.
$$
Cela conduit à la formule :
$$
d^k = -\left[B^k\right]^{-1} \nabla f(x^k) = - H^k g^k. 
$$

On cherche ensuite $\alpha_k$ satisfaisant (5) par la méthode de ``backtracking", toujours avec $\alpha=1$ et on pose
$$
x^{k+1} := x^k +\alpha_k d^k.
$$

Maintenant, nous avons besoin de calculer approximation $H^{k+1}$ de $\left[D^2f(x^{k+1})\right]^{-1}$. Pour cela, rappelons que nous voulons
$$
\tilde m_{k+1} (d):= f(x^{k+1}) + g^{k+1}\cdot d +\dfrac 12 d^T B^{k+1} d,
$$
soit une approximation de
$$
\overline m_{k+1}(d):= f(x^{k+1} + d).
$$
Nous avons déjà par construction,
$$
\tilde m_{k+1}(0)=\overline m_{k+1}(0)=f(x^{k+1})\qquad\text{et}\qquad \nabla \tilde m_{k +1}(0)=\nabla \overline m_{k+1}(0)=g(x^{k+1}).
$$
Nous appliquons la nouvelle condition
$$
\nabla m_{k+1}(-\tau_k d^k)=\nabla \overline m_{k+1}(-\tau_k d^k)=g^k.
$$

En notant $a^k:=g^{k+1}-g^k$ et $b^k:=\tau^kd^k=x^{k+1}-x^k$, cela équivaut à $B^{k+1}b^k=a^k$. En supposant que $B^{k+1}$ est inversible, cela équivaut à demander que $H^{k+1}$ soit solution de
$$
\tag{6}
Ha^k=b^k.
$$
Une condition nécessaire et suffisante pour que (6) admette une solution symétrique définie positive $H$ est :
$$
\tag{7}
\left<a^k;b^k\right> >0.
$$

Nous ne voulons pas perdre toute l'information déjà contenue dans $H^k$, donc, en supposant que (7) soit vraie, nous choisissons une solution de (6) aussi proche que possible de $H^k$. Un choix populaire consiste à définir :
$$
\tag{8}
H^{k+1} := \left(I-\rho_k b^k\otimes a^k\right) H^k \left(I-\rho_k a^k\otimes b^k\right) + \rho_k b^k\otimes b^k,\quad\text{ avec }\quad \rho_k:=\dfrac1{\left<a^k;b^k\right>}.
$$

__Question 18.__ Vérifiez que la formule (8) donne bien une solution à (6). Vérifiez que $H^{k+1}$ ainsi définie est une matrice symétrique définie positive 

___Solution:___
En supposant que $H^k$ soit définie positive et que (7) soit vraie, il est facile de vérifier que $H^{k+1}$ est également définie positive. Tout d'abord, pour $d\in \mathbb{R}^N$, en notant $w=d-\rho_k \left<b^k;d\right>a^k$, nous calculons,

$$
  d^TH^{k+1}d = w^TH^kw + \rho_k \left<d;b^k\right>^2\ \geq \ 0.
$$

Ensuite, en utilisant la même formule, nous voyons que $d^TH^{k+1}d=0$ implique : (a) $w=0$ et (b) $\left<d;b^k\right>=0$. Le point (a) entraîne que $d=\lambda a^k$ pour un $\lambda\in \mathbb{R}$ et avec (b) on déduit $\lambda\left<a^k;b^k\right>=0$. Donc $\lambda=0$ et $d=0$. Cela prouve que $H^{k+1}$ est définie positive.  

__Question 19.__ Implémentez la méthode BFGS et appliquez-la à la fonction ci-dessus avec $c = 0.1$, $\beta=0.75$ et $x^0=(0,0)$. Comme premier approximation de $D^2f(x^0)$ on prendra $H^0=I$.

Représentez les itérations sur un graphique et tracez $\ \log(f(x^k))\ $ en fonction de $k$. Observez et commentez.

__Question 20.__ Est-ce que $H^k$ converge vers $[D^2 f(x^*)]^{-1}$ ?

In [ ]:
## Data of the optimization problems
Lambda = 1000
f = lambda x,y : ( x - 1)**2 + Lambda*(y - x**2)**2
df = lambda x,y : np.array([2*(x - 1) + 4*Lambda*x*(x**2 - y), 2*Lambda*(y - x**2)])
ddf = lambda x,y : np.array([[2 - 4*Lambda*y + 12*Lambda*x**2 , 
                              -4*Lambda*x],[-4*Lambda*x, 2*Lambda]])
HH = lambda x,y : (1/(4*Lambda - 8*(Lambda**2)*y 
                    + 8*(Lambda**2)*x**2))*np.array([[2*Lambda, 4*Lambda*x],
                    [4*Lambda*x, 2 - 4*Lambda*y + 12*Lambda*x**2 ]])

In [ ]:
## Parameters of the algorithm
c, beta = .1, .75
epsilon = 1e-8
nitermax = 200
niter_lsmax = 50

In [ ]:
## Initialization 
I = np.identity(2)
x, y = 0, .5
w = np.array([x, y])

fw = f(x, y)
g = np.array(df(x, y))
ng = np.hypot(g[0], g[1])
ng0=ng
H = np.identity(2)

niter = 0
flag = 'OK'

W, F, NG, Alpha =[w], [fw], [ng], [] 
ErrH = [np.linalg.norm(H-HH(x,y), ord = 'fro')]

In [ ]:
## Optmization loop
while (niter < nitermax):
    if ng < epsilon*ng0 or flag =='Not OK': break
    d = -H@g
    dx, dy = d[0], d[1]
    new_fw = f(x + dx, y + dy) 
    alpha, niter_ls = 1, 0
    fact = c*np.dot(d,g)
    while (new_fw - fw - alpha*fact>=0):
        alpha *= beta
        new_fw = f(x + alpha*dx, y + alpha*dy)
        niter_ls += 1
        if (niter_ls>=niter_lsmax):
            flag = 'Not OK'
            break
    x, y, fw = x + alpha*dx, y + alpha*dy, new_fw  
    old_g, g = g, np.array(df(x, y)) 
    ng = np.hypot(g[0], g[1])
    a, b = g - old_g, np.array([alpha*dx, alpha*dy])
    a_dot_b = np.dot(a,b)
    if a_dot_b<=0:
        flag = 'Not OK'
        break
    rho=1/a_dot_b
    H = np.dot(np.dot( 
        I - rho*np.tensordot(b,a,0), H), 
        I - rho*np.tensordot(a,b,0) 
        ) + rho*np.tensordot(b,b,0) 
    
    ErrH.append(np.linalg.norm(H-HH(x,y)
                        /np.linalg.norm(HH(x,y)), ord = 'fro'))
    W.append(np.array([x, y]))
    F.append(fw)
    NG.append(ng)
    Alpha.append(alpha)
    niter += 1

In [ ]:
print(f"flag = {flag}, nombre d'itérations = {niter}")    
    
# plot the results 
xmin, xmax, ymin, ymax = -.8, 1.8, -.5, 1.5
plt.figure()

import matplotlib.cm as cm
import itertools as it
l =len(W)
colors = it.cycle(cm.rainbow(np.linspace(0, 1, l)))
for w in W :
    plt.plot(w[0], w[1], '.', color=next(colors))
W = np.array(W)
plt.plot(W[:,0],W[:,1],linestyle=':')    
level_lines(f, xmin, xmax, ymin, ymax, np.linspace(1, 100, 5))
draw_vector_field(df, xmin, xmax, ymin, ymax, 15)
plt.title(r'Les itérés $w^k$', fontsize=15)
plt.xlabel(r'x')
plt.ylabel(r'y')
plt.axis('equal')
plt.show()

# plot of the values of f along the iterations.
plt.figure()
plt.semilogy(range(len(F)),F,'.',linestyle=':')
plt.title(r"$f(w^k)$ en fonction des itérations", fontsize=15)
plt.show()

# plot of the values of norm (grad f) along the iterations.
plt.figure()
plt.semilogy(range(len(NG)),np.array(NG)/ng0,'.',linestyle=':')
plt.title(r"$\|\nabla f(w^k)\|/\|\nabla f(w^0)\|$ en fonction des itérations", fontsize=15)
plt.show()

# plot of the values of ||Happrox-Htrue|| along the iterations.
plt.figure()
plt.semilogy(range(len(ErrH)),ErrH,'.',linestyle=':')
plt.title(r'$|H^k - [D^2f(w^k)]^{-1}|$')
plt.xlabel('niter')
plt.show()

# plot of the step alpha along the iterations.
plt.figure()
plt.plot(range(len(Alpha)),Alpha,'.')
plt.title(r"$\alpha$ en fonction des itérations")
plt.xlabel('niter')
plt.show()

On observe à nouveau deux comportement successifs de la convergence. Une convergence relativemente puis une convergence très rapide en 4 ou cinq pas qui est concomitante de l'acceptation de $\alpha=1$ par le critère d'Armijo.

La méthode nécessite deux fois plus d'itérations que la méthode de Newton pour la précision demandée. Cependant, ici on n'a pas à résoudre le système linéaire $D^2f(w^k) d^k =-g^k$ à chaque itération. Cette résolution (très coûteuse pour les grands systèmes mal conditionnés) est le défaut de la méthode de Newton.

La matrice $H^k$ "approchant" $[D^2f(w^k)]^{-1}$ ne converge pas vers $[D^2f(w^*)]^{-1}$. En fait, on a seulement une approximation dans la direction $w^k-w^*$, 
$$
H^k(w^k-w^*)\simeq [D^2f(w^*)]^{-1}(w^k-w^*).
$$